In [7]:
# Test env
# Create a folder to store result
import os, pathlib
from dotenv import load_dotenv


pathlib.Path("Out").mkdir(exist_ok=True)
print("Out/ ready.")

load_dotenv()
print("Env loaded. GOOGLE_API_KEY present? ", bool(os.getenv("GOOGLE_API_KEY")))
print("Kernel OK.")

Out/ ready.
Env loaded. GOOGLE_API_KEY present?  True
Kernel OK.


In [8]:
# Test API Keys
API_KEY = os.getenv("GOOGLE_API_KEY")
CX_ID = os.getenv("GOOGLE_CX_ID")
assert API_KEY and CX_ID, "Set GOOGLE_API_KEY and GOOGLE_CX_ID in your .env file"
print("OK, keys loaded.")

OK, keys loaded.


In [9]:
# Setup Queries and Pages
QUERY = "site:instagram.com nail Tokyo"
PAGES = 5

In [10]:
import re
from urllib.parse import urlparse


def username_from_url(url: str):
    """
    Accept only top-level profile URLs, ignore posts/reels/stories.
    """
    p = urlparse(url)
    parts = [seg for seg in p.path.split("/") if seg]
    if not parts:
        return None
    if parts[0].lower() in {"p", "reel", "stories", "explore"}:
        return None
    return parts[0] if re.fullmatch(r"[A-Za-z0-9._]+", parts[0]) else None


# Handles: "12,345 followers", "2.1k followers", "3m followers"
# and JP formats like "フォロワー 1.2万"
FOLLOWER_PATTERNS = [
    r"([\d.,]+)\s*(k|m|b)?\s+followers",
    r"followers:\s*([\d.,]+)\s*(k|m|b)?",
    r"フォロワー\s*([\d.,]+)\s*(万|千)?",
]

def parse_followers(snippet: str):
    text = (snippet or "").lower()
    for pat in FOLLOWER_PATTERNS:
        m = re.search(pat, text)
        if not m:
            continue
        num = m.group(1).replace(",", "")
        unit = (m.group(2) or "").lower()
        try:
            val = float(num)
        except ValueError:
            continue
        mult = 1
        if unit in {"k", "千"}: mult = 1_000
        elif unit == "m":       mult = 1_000_000
        elif unit == "b":       mult = 1_000_000_000
        elif unit == "万":      mult = 10_000
        return int(val * mult)
    # Loose fallback like "12k" without the word "followers"
    m2 = re.search(r"\b([\d.,]+)\s*(k|m|b)\b", text)
    if m2:
        val = float(m2.group(1).replace(",", ""))
        mult = {"k":1_000,"m":1_000_000,"b":1_000_000_000}[m2.group(2)]
        return int(val * mult)
    return None

In [11]:
import requests


def google_page(query: str, start: int = 1):
    r = requests.get(
        "https://www.googleapis.com/customsearch/v1",
        params={"key": API_KEY, "cx": CX_ID, "q": query, "start": start},
        timeout=30
    )
    r.raise_for_status()
    return r.json()

data = google_page(QUERY, start=1)
len(data.get("items", []) or [])

10

In [12]:
items = data.get("items", []) or []
rows = []

for it in items:
    link = it.get("link", "")
    if "instagram.com" not in link:
        continue
    uname = username_from_url(link)
    if not uname:
        continue

    snippet = (it.get("snippet") or "").strip()
    followers = parse_followers(snippet)

    rows.append({
        "username": uname,
        "followers": followers,
        "bio": snippet,
        "profile_url": f"https://www.instagram.com/{uname}/"
    })

rows[:5]

[{'username': 'nail.sucre',
  'followers': 94000,
  'bio': '94K followers · 22 following · 10012 posts · @nail.sucre: “新宿区新宿3丁目3-9新宿伍名館204 TOKYO.SHINJUKU @nail.sucreamer @sucre.newyork sister store No\xa0...',
  'profile_url': 'https://www.instagram.com/nail.sucre/'},
 {'username': 'hanna_nailwizard',
  'followers': None,
  'bio': 'ACCEPTING AUG/SEP BOOKING LA BASED Currently in TOKYO Setagaya English/Jpnse speaking NailTech DM for appointment #englishspeakingnailsalon. Follow.',
  'profile_url': 'https://www.instagram.com/hanna_nailwizard/'},
 {'username': 'reina.tokyonailart',
  'followers': None,
  'bio': "Japanese nail tech ߉ #SomervilleNJ. Life's too short for boring nails.✶ Nail reservation,collab ▶︎DM me Founder of @hitomebore.tokyo · reinatokyonailart.",
  'profile_url': 'https://www.instagram.com/reina.tokyonailart/'},
 {'username': 'chirin._chan',
  'followers': None,
  'bio': "I'm Chirin, a nail artist at Hakusan Station, I'm happy to create beautiful, customized nail art fo

In [13]:
rows[:]

[{'username': 'nail.sucre',
  'followers': 94000,
  'bio': '94K followers · 22 following · 10012 posts · @nail.sucre: “新宿区新宿3丁目3-9新宿伍名館204 TOKYO.SHINJUKU @nail.sucreamer @sucre.newyork sister store No\xa0...',
  'profile_url': 'https://www.instagram.com/nail.sucre/'},
 {'username': 'hanna_nailwizard',
  'followers': None,
  'bio': 'ACCEPTING AUG/SEP BOOKING LA BASED Currently in TOKYO Setagaya English/Jpnse speaking NailTech DM for appointment #englishspeakingnailsalon. Follow.',
  'profile_url': 'https://www.instagram.com/hanna_nailwizard/'},
 {'username': 'reina.tokyonailart',
  'followers': None,
  'bio': "Japanese nail tech ߉ #SomervilleNJ. Life's too short for boring nails.✶ Nail reservation,collab ▶︎DM me Founder of @hitomebore.tokyo · reinatokyonailart.",
  'profile_url': 'https://www.instagram.com/reina.tokyonailart/'},
 {'username': 'chirin._chan',
  'followers': None,
  'bio': "I'm Chirin, a nail artist at Hakusan Station, I'm happy to create beautiful, customized nail art fo

In [ ]:
print("Hello World!")
